# **15.6 Class_Methods_and_Static_Methods**

Not all methods need `self`. Class methods operate on the class itself (using `@classmethod`), while static methods are just utility functions bundled with the class (using `@staticmethod`). In this lesson you'll learn when and how to use these alternative method types to create factory methods, alternative constructors, and class-level utilities for your Pokemon classes.

---

## **Regular vs Class vs Static Methods**

Three types of methods, each with different first parameters.

In [ ]:
class Pokemon:
    species_count = 0  # Class variable
    
    def __init__(self, name, level):
        self.name = name
        self.level = level
        Pokemon.species_count += 1
    
    # INSTANCE METHOD — takes self
    def display(self):
        return f"{self.name} Lv.{self.level}"
    
    # CLASS METHOD — takes cls
    @classmethod
    def get_count(cls):
        return f"Total Pokemon created: {cls.species_count}"
    
    # STATIC METHOD — takes neither
    @staticmethod
    def is_valid_level(level):
        return 1 <= level <= 100

pikachu = Pokemon("Pikachu", 25)
charizard = Pokemon("Charizard", 36)

# Instance method — needs instance
print(pikachu.display())

# Class method — can call on class
print(Pokemon.get_count())

# Static method — utility function
print(f"Is level 150 valid? {Pokemon.is_valid_level(150)}")
print(f"Is level 50 valid? {Pokemon.is_valid_level(50)}")

---

## **@classmethod for Alternative Constructors**

Class methods are perfect for factory methods — alternative ways to create instances.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp, attack):
        self.name = name
        self.level = level
        self.hp = hp
        self.attack = attack
    
    @classmethod
    def from_string(cls, pokemon_string):
        """Create from 'Name,Level,HP,Attack' format."""
        name, level, hp, attack = pokemon_string.split(',')
        return cls(name, int(level), int(hp), int(attack))
    
    @classmethod
    def starter(cls, name, starter_type):
        """Create a starter Pokemon."""
        stats = {
            "Fire": (39, 52),    # hp, attack
            "Water": (44, 48),
            "Grass": (45, 49),
        }
        hp, attack = stats[starter_type]
        return cls(name, 5, hp, attack)
    
    def __str__(self):
        return f"{self.name} Lv.{self.level} (HP:{self.hp} ATK:{self.attack})"

# Normal constructor
pikachu = Pokemon("Pikachu", 25, 35, 55)
print(pikachu)

# Alternative constructor from string
charizard = Pokemon.from_string("Charizard,36,78,84")
print(charizard)

# Alternative constructor for starters
squirtle = Pokemon.starter("Squirtle", "Water")
print(squirtle)

---

## **@staticmethod for Utilities**

Static methods are regular functions that happen to live in the class namespace.

In [ ]:
class Pokemon:
    def __init__(self, name, level):
        self.name = name
        self.level = level
    
    @staticmethod
    def calculate_damage(attacker_level, move_power, defender_defense):
        """Calculate damage — doesn't need instance or class."""
        return (attacker_level * move_power) // defender_defense
    
    @staticmethod
    def is_type_effective(attacker_type, defender_type):
        """Check type effectiveness."""
        effectiveness = {
            ("Electric", "Water"): 2.0,
            ("Fire", "Grass"): 2.0,
            ("Water", "Fire"): 2.0,
        }
        return effectiveness.get((attacker_type, defender_type), 1.0)

# Use without creating an instance
damage = Pokemon.calculate_damage(25, 90, 40)
print(f"Damage: {damage}")

multiplier = Pokemon.is_type_effective("Electric", "Water")
print(f"Effectiveness: {multiplier}x")

---

## **Class Methods Accessing Class State**

Class methods can read and modify class variables.

In [ ]:
class Pokemon:
    _registry = {}  # Class variable
    
    def __init__(self, name, pokedex_id):
        self.name = name
        self.pokedex_id = pokedex_id
        Pokemon._registry[pokedex_id] = self
    
    @classmethod
    def get_by_id(cls, pokedex_id):
        """Look up Pokemon by ID from class registry."""
        return cls._registry.get(pokedex_id)
    
    @classmethod
    def list_all(cls):
        """List all registered Pokemon."""
        return list(cls._registry.values())
    
    @classmethod
    def clear_registry(cls):
        """Reset the registry."""
        cls._registry.clear()
    
    def __str__(self):
        return f"#{self.pokedex_id:03d} {self.name}"

# Create Pokemon — automatically registered
Pokemon("Bulbasaur", 1)
Pokemon("Ivysaur", 2)
Pokemon("Venusaur", 3)

# Look up by ID
bulbasaur = Pokemon.get_by_id(1)
print(f"Found: {bulbasaur}")

# List all
print("\nAll Pokemon:")
for p in Pokemon.list_all():
    print(f"  {p}")

---

## **When to Use Each**

Guidelines for choosing the right method type.

In [ ]:
guidelines = """
Use INSTANCE METHOD when:
- Need access to instance data (self.attribute)
- Modifying instance state
- Most common type

Use CLASS METHOD when:
- Alternative constructors/factory methods
- Need access to class variables
- Operating on the class itself
- Want to work with subclasses correctly

Use STATIC METHOD when:
- Don't need instance or class data
- Utility function that logically belongs with the class
- Could be a standalone function but you want it in the class namespace
"""
print(guidelines)

---

## **Practical: Complete Pokemon Registry**

In [ ]:
class Pokemon:
    """Complete Pokemon with instance, class, and static methods."""
    
    _registry = {}  # Class-level registry
    total_created = 0  # Class-level counter
    
    def __init__(self, name, ptype, level, hp):
        self.name = name
        self.type = ptype
        self.level = level
        self.hp = hp
        
        Pokemon.total_created += 1
        Pokemon._registry[name] = self
    
    # INSTANCE METHODS
    def display(self):
        return f"{self.name} ({self.type}) Lv.{self.level}"
    
    def level_up(self):
        self.level += 1
        self.hp += 5
    
    # CLASS METHODS — Alternative constructors
    @classmethod
    def from_dict(cls, data):
        """Create from dictionary."""
        return cls(data['name'], data['type'], data['level'], data['hp'])
    
    @classmethod
    def starter(cls, name, starter_type):
        """Create starter Pokemon."""
        return cls(name, starter_type, 5, 40)
    
    # CLASS METHODS — Registry access
    @classmethod
    def find(cls, name):
        """Find Pokemon by name in registry."""
        return cls._registry.get(name)
    
    @classmethod
    def get_stats(cls):
        """Get class-level statistics."""
        return {
            'total_created': cls.total_created,
            'in_registry': len(cls._registry)
        }
    
    # STATIC METHODS — Utilities
    @staticmethod
    def calculate_exp_needed(level):
        """Calculate experience needed for level."""
        return level ** 3
    
    @staticmethod
    def is_valid_type(ptype):
        """Check if type is valid."""
        valid = {"Fire", "Water", "Grass", "Electric", "Normal"}
        return ptype in valid
    
    def __str__(self):
        return self.display()

# Test all method types
print("=" * 50)
print("Testing Pokemon Class")
print("=" * 50)

# Instance method
pikachu = Pokemon("Pikachu", "Electric", 25, 35)
print(f"\nInstance method: {pikachu.display()}")

# Class method — alternative constructor
charizard_data = {'name': 'Charizard', 'type': 'Fire', 'level': 36, 'hp': 78}
charizard = Pokemon.from_dict(charizard_data)
print(f"From dict: {charizard}")

squirtle = Pokemon.starter("Squirtle", "Water")
print(f"Starter: {squirtle}")

# Class method — registry
found = Pokemon.find("Pikachu")
print(f"\nFound in registry: {found}")

stats = Pokemon.get_stats()
print(f"Stats: {stats}")

# Static method — utilities
exp = Pokemon.calculate_exp_needed(50)
print(f"\nExp for level 50: {exp}")

print(f"Is 'Dragon' valid? {Pokemon.is_valid_type('Dragon')}")
print(f"Is 'Fire' valid? {Pokemon.is_valid_type('Fire')}")

---

## **Summary**

- Instance methods take `self` — operate on instances
- Class methods take `cls` — operate on the class
- Static methods take neither — utility functions
- `@classmethod` for alternative constructors
- `@classmethod` for accessing class variables
- `@staticmethod` for related utility functions
- Class methods work correctly with inheritance